In [1]:
import pandas as pd
from ipywidgets import widgets
import matplotlib.pylab as plt
from IPython.display import clear_output

In [7]:
data = pd.read_excel('verbali-esami.xls')
selected = data
#data.iloc[:10,3:8]

In [8]:
insegnamenti = sorted(frozenset(data['Descrizione insegnamento']))
appelli = lambda insegnamento: sorted(frozenset(data[data['Descrizione insegnamento'] == insegnamento].Data.dt.date))
verbali = lambda insegnamento, appello: data[(data['Descrizione insegnamento'] == insegnamento) & (data.Data.dt.date == appello)]
starts_with_digit = lambda _: _[0] in frozenset('0123456789')
ends_with_digit = lambda _: _[-1] in frozenset('0123456789')
perc = lambda _: (10000*_/sum(_)).astype(int)/100
count = lambda _: pd.DataFrame(_.value_counts().sort_index()).T

def hist(_):
    perc(_.value_counts()).sort_index().plot.bar()

def euniq(e):
    v = list(filter(starts_with_digit, e.values))
    if v: return v[0]
    v = list(filter(ends_with_digit, e.values))
    if v: return max(v)
    if 'Respinto' in e.values: return 'RE'
    if 'Ritirato' in e.values: return 'RI'   
    
def emerge(e):
    res = []
    for v in e.values:
        if starts_with_digit(v) or ends_with_digit(v): res.append(v)
        if v == 'Respinto': res.append('RE')
        if v == 'Ritirato': res.append('RI')
    return tuple(res)    

def emerge_sempl(e):
    res = []
    for v in e.values:
        if starts_with_digit(v): res.append('OK')
        if v[0] == '~': res.append('~')
        if v == 'Respinto': res.append('RE')
        if v == 'Ritirato': res.append('RI')
    return tuple(res)

In [9]:
def update_data():

    esiti = selected.sort_values('Data').apply(
        lambda r: pd.Series({'Matricola': r.Matricola, 'Esito': ('~' + r.Voto if r['Stato Esito'] == 'Rifiutato' else r.Voto)}), axis = 1
    )[['Matricola', 'Esito']]

    esiti_unici = esiti.groupby('Matricola').agg(euniq).Esito

    esiti_unici_sempl = esiti_unici.apply(
        lambda _: 'OK' if starts_with_digit(_) else ('~' if _[0] == '~' else _)
    )

    esiti_unici_promossi = esiti_unici[esiti_unici.apply(starts_with_digit)].apply(
        lambda _: 33 if _ == '30 e lode' else int(_)
    )

    patterns = esiti.groupby('Matricola').agg(emerge)

    patterns_sempl = esiti.groupby('Matricola').agg(emerge_sempl)
    
    patterns_len = patterns[patterns.Esito.apply(lambda _: starts_with_digit(_[-1]))].Esito.apply(len)

    with w0:
        clear_output()
        display(count(esiti_unici))
        display(count(esiti_unici_sempl))

    with wo00:
        clear_output()
        hist(esiti_unici_sempl)
        plt.show()

    with wo01:
        clear_output()
        hist(esiti_unici_promossi)
        plt.show()
        
    with wo10:
        clear_output()
        hist(patterns_len)
        plt.show()

    with wo11:    
        clear_output()
        patterns_sempl.Esito.apply(' '.join).value_counts(ascending = True).tail(10).plot.barh()
        plt.show()

In [10]:
wins = widgets.Select(description = 'Insegnamenti')
wdate = widgets.SelectMultiple(description = 'Date')
wlabel = widgets.HTML(description = 'Selezione')

w0 = widgets.Output()
wo00 = widgets.Output()
wo01 = widgets.Output()
wo10 = widgets.Output()
wo11 = widgets.Output()

def uwdate(change):
    wdate.options = appelli(change.new)
    wdate.value = wdate.options
wins.observe(uwdate, names = 'value')

def uselected(change):
    global selected
    selected = data[(data['Descrizione insegnamento'] == wins.value) & (data.Data.dt.date.isin(wdate.value))]
    wlabel.value = "<b>Verbali</b>: {}<br/><b>Iscritti</b>: {}".format(len(selected), len(set(selected.Matricola)))
    update_data()
    
wdate.observe(uselected, names = 'value')    

wins.options = insegnamenti
wins.value = insegnamenti[0]
wdate.options = wdate.value = appelli(insegnamenti[0])

display(widgets.VBox([
    widgets.HBox([wins, wdate, wlabel]),
    w0,
    widgets.HBox([wo00, wo01]),
    widgets.HBox([wo10, wo11])
]))  

KeyError: "['Esito'] not in index"